Aim: Predict if PM concentration is higher than boundary value based on meteorologic parameters, leave out influence of traffic data.
daily boundary value for PM10:  50 µg/m³

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt

%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

In [128]:
os.chdir(r"C:\Users\Anna\Documents\UNI\MA Semi 1\Machine Learning\Final_project")

## test file from luftdateninfo
file_pm = "test_PM_2020-01-23_sds011_sensor_10963.csv"
pm_ds = pd.read_csv(file_pm, sep=";", index_col="timestamp")

## df with time_col as index
pm_ds = pm_ds[["P1"]]

#Sum up to hourly values
pm_ds.index = pd.to_datetime(pm_ds.index) # convert index into time_dtype

pm_ds_hour = pm_ds.groupby(pm_ds.index.hour).mean()# ds= one day -->24 rows
pm_ds_hour.index = pd.to_datetime(pm_ds_hour.index, unit="h")
print(pm_ds_hour)


                            P1
timestamp                     
1970-01-01 00:00:00  61.175455
1970-01-01 01:00:00  60.337778
1970-01-01 02:00:00  60.901000
1970-01-01 03:00:00  58.587727
1970-01-01 04:00:00  53.806000
1970-01-01 05:00:00  42.638333
1970-01-01 06:00:00  42.976667
1970-01-01 07:00:00  45.372917
1970-01-01 08:00:00  43.117500
1970-01-01 09:00:00  46.309130
1970-01-01 10:00:00  38.529583
1970-01-01 11:00:00  33.917500
1970-01-01 12:00:00  31.267083
1970-01-01 13:00:00  28.494000
1970-01-01 14:00:00  26.772917
1970-01-01 15:00:00  25.041667
1970-01-01 16:00:00  23.417917
1970-01-01 17:00:00  32.340417
1970-01-01 18:00:00  42.986818
1970-01-01 19:00:00  40.723913
1970-01-01 20:00:00  43.877083
1970-01-01 21:00:00  47.411250
1970-01-01 22:00:00  48.723333
1970-01-01 23:00:00  38.984737


Load meteorologic test-file and merge both ds (PM + meteo)

In [149]:
file_wind =  pd.read_csv("./DWDstats_meteo/produkt_ff_stunde_19530101_20191231_04931.txt", sep=";", header=0,
                         index_col="MESS_DATUM")
#print(file_wind)
file_rain = pd.read_csv("./DWDstats_meteo/produkt_rr_stunde_19951004_20191231_04931.txt", sep=";", header=0,
                         index_col="MESS_DATUM")
file_pressure = pd.read_csv("./DWDstats_meteo/produkt_p0_stunde_19490101_20191231_04931.txt", sep=";", header=0,
                          index_col="MESS_DATUM") 
file_temperature = pd.read_csv("./DWDstats_meteo/produkt_tu_stunde_19880101_20191231_04931.txt", sep=";", header=0,
                          index_col="MESS_DATUM") 

allmeteo = file_wind.merge(file_rain, on ="MESS_DATUM").merge(file_pressure, on ="MESS_DATUM").merge(file_temperature, on ="MESS_DATUM")
allmeteo = allmeteo[["   F", "   D", "  R1", "  P0", "TT_TU", "RF_TU"]]
allmeteo = allmeteo.rename(columns={"   F": "F", "   D":"D", "  R1": "R1", "  P0": "P0"})

print(allmeteo)


####################
#import glob
#files_meteo = glob.glob('./DWDstats_meteo/*04931.txt')

# allmeteo_ds = 
# for file in files_meteo:
#     meteo_ds_hour = pd.read_csv(file, sep=";", index_col="MESS_DATUM")
#     print(meteo_ds_hour)
    
#df1.merge(df2, on ="MESS_DATUM")
####################################

# ## df with time_col as index
# pm_ds = pm_ds[["P1"]]

# #Sum up to hourly values
# pm_ds.index = pd.to_datetime(pm_ds.index) # convert index into time_dtype



              F    D   R1     P0  TT_TU  RF_TU
MESS_DATUM                                    
1995100414  2.6  220  0.0  970.6   23.3   51.0
1995100415  3.6  210  0.0  970.3   23.8   50.0
1995100416  1.5  150  0.0  969.9   23.0   54.0
1995100417  1.5  120  0.0  970.0   21.5   63.0
1995100418  1.5  180  0.0  970.1   19.4   71.0
...         ...  ...  ...    ...    ...    ...
2019123119  1.3  270  0.0  987.3    0.3   82.0
2019123120  0.9  270  0.0  987.8    0.9   84.0
2019123121  1.0  230  0.0  987.2    1.2   87.0
2019123122  0.9  300  0.0  987.5    1.4   87.0
2019123123  0.5  230  0.0  987.3    1.1   89.0

[211931 rows x 6 columns]


In [155]:
allmeteo2 = allmeteo
## convert both indeces to same datetime
## style: YYYY-MM-DDTHH:MM:SS2020-01-23T23:47:57
print
allmeteo2.index = pd.to_datetime(allmeteo2.index,format="%Y%m%d%H", unit="ns")
#
#allmeteo.index = pd.to_datetime(allmeteo.index, unit="h")
print(allmeteo2.index)


# ### merge both DS (meteo Ds + PM DS)
# print(type(allmeteo.index))
# PM_meteo = pm_ds_hour.merge(allmeteo, left_index = "timestamp", right_index="MESS_DATUM")
# PM_meteo

DatetimeIndex(['1970-01-01 00:00:01.995100414',
               '1970-01-01 00:00:01.995100415',
               '1970-01-01 00:00:01.995100416',
               '1970-01-01 00:00:01.995100417',
               '1970-01-01 00:00:01.995100418',
               '1970-01-01 00:00:01.995100419',
               '1970-01-01 00:00:01.995100420',
               '1970-01-01 00:00:01.995100421',
               '1970-01-01 00:00:01.995100422',
               '1970-01-01 00:00:01.995100423',
               ...
               '1970-01-01 00:00:02.019123114',
               '1970-01-01 00:00:02.019123115',
               '1970-01-01 00:00:02.019123116',
               '1970-01-01 00:00:02.019123117',
               '1970-01-01 00:00:02.019123118',
               '1970-01-01 00:00:02.019123119',
               '1970-01-01 00:00:02.019123120',
               '1970-01-01 00:00:02.019123121',
               '1970-01-01 00:00:02.019123122',
               '1970-01-01 00:00:02.019123123'],
              dtype=

In [ ]:
#####################################

In [ ]:
# split the boston hoursing dataset into training and testing 

ratio = 0.2 # split ratio
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=ratio,random_state=5)

#print the size of train and test dataset

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


In [ ]:
softReg = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs')
softReg.fit(Xstan,y)



In [ ]:
softReg.intercept_,softReg.coef_


In [ ]:
yhat = softReg.predict(Xstan)
dataStan['predict'] = yhat
dataStan.head()


Evaluation of Softmax permorfance

In [ ]:
C = confusion_matrix(dataStan['category'].values,yhat)
confusionMatrix = pd.DataFrame(data = C, index=['poor(0), true','good(1), true','great(2), true'], columns = ['poor(0), predicted','good(1), predicted','great(2), predicted'])
confusionMatrix.loc['sum'] = confusionMatrix.sum()
confusionMatrix['sum'] = confusionMatrix.sum(axis=1)
confusionMatrix
